In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Reshape, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.optimizers import Adam, SGD
import math, time, cv2, itertools
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pdb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('../input/train.csv')
# test_data = pd.read_csv('../input/test.csv').values

In [ ]:
y = pd.get_dummies(train_data.label).values
X = train_data.iloc[:, 1:].values
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.9, random_state = 0)
X_tr, X_val, y_tr, y_val = train_test_split(train_X, train_y, train_size=0.8, random_state = 0)
print(type(X_tr), type(y_tr), type(X_val), type(y_val), type(test_X), type(test_y))
print(X_tr.shape, y_tr.shape, X_val.shape, y_val.shape, test_X.shape, test_y.shape)

In [ ]:
def next_batch(step, batch_size, data, labels):
    return data[step*batch_size:(step + 1)*batch_size], labels[step*batch_size:(step + 1)*batch_size]

 ## Simple linear model

In [ ]:
x_input = tf.placeholder(tf.float32, shape=[None, 784])
y_label = tf.placeholder(tf.float32, shape=[None, 10])

#W = tf.Variable(tf.zeros([784, 10]))
#b = tf.Variable(tf.zeros([10]))

W = tf.Variable(tf.truncated_normal([784, 10], stddev=0.1))
b = tf.Variable(tf.constant(0.1, shape=[10]))

y_pred = tf.matmul(x_input, W) + b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_label, logits=y_pred))

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
epochs = 10
batch_size = 100
steps = math.ceil(train_X.shape[0] // batch_size)

train_accs = []
eval_accs = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        for step in range(steps):
            batch_X, batch_y = next_batch(step, batch_size, train_X, train_y)
            #if step % 50 == 0:
            #    train_accuracy = accuracy.eval(feed_dict={x_input: batch_X, y_label: batch_y})
            #    print('step %d, training accuracy %g' % (step, train_accuracy))
            train_step.run(feed_dict={x_input: batch_X, y_label: batch_y})
        train_accuracy = accuracy.eval(feed_dict={x_input: batch_X, y_label: batch_y})
        eval_accuracy = accuracy.eval(feed_dict={x_input: val_X, y_label: val_y})
        train_accs.append(train_accuracy)
        eval_accs.append(eval_accuracy)
        print('Epoch: %d - Accuracy: %g' % (epoch, eval_accuracy))

x = np.array(range(epochs))
plt.plot(x, train_accs)
plt.plot(x, eval_accs)
plt.show()

## TensorFlow Convolutional Network

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
# Input
x_input = tf.placeholder(tf.float32, shape=[None, 784])
y_label = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x_input, [-1, 28, 28, 1])

# Convoution layer 1
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Convoution layer 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# FC layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Softmax layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_label, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

epochs = 30
batch_size = 100
steps = math.ceil(train_X.shape[0] // batch_size)

train_accs = []
eval_accs = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        for step in range(steps):
            batch_X, batch_y = next_batch(step, batch_size, train_X, train_y)
#             if step % 50 == 0:
#                 train_accuracy = accuracy.eval(feed_dict={x_input: batch_X, y_label: batch_y, keep_prob: 1.0})
#                 print('step %d, training accuracy %g' % (step, train_accuracy))
            train_step.run(feed_dict={x_input: batch_X, y_label: batch_y, keep_prob: 0.5})
        train_accuracy = accuracy.eval(feed_dict={x_input: batch_X, y_label: batch_y, keep_prob: 1.0})
        eval_accuracy = accuracy.eval(feed_dict={x_input: X_val, y_label: y_val, keep_prob: 1.0})
        print('Epoch %d, val accuracy %g' % (epoch, eval_accuracy))
        train_accs.append(train_accuracy)
        eval_accs.append(eval_accuracy)
    accuracy_l = accuracy.eval(feed_dict={x_input: test_X, y_label: test_y, keep_prob: 1.0})
    print('test accuracy ', accuracy_l)
        
x = np.array(range(epochs))
plt.plot(x, train_accs)
plt.plot(x, eval_accs)
plt.show()

## Keras model

In [ ]:
model = Sequential()
model.add(Reshape((28, 28, 1), input_shape=(784,)))
model.add(Conv2D(32, (5, 5), padding='same', activation='relu', ))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

In [ ]:
epochs = 20
batch_size = 100
model.fit(X_tr, y_tr,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val)
         )

In [ ]:
score = model.evaluate(test_X, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])